In [ ]:

#imports 
import pandas as pd; pd.set_option('display.max_columns', None), pd.set_option('display.max_rows', None)
import math
import numpy as np

#importing all csvs and storing into unique variables 
ccm_1 = pd.read_csv('0701ccm1.csv')
ccm_2 = pd.read_csv('0701ccm2.csv')
ccm_3 = pd.read_csv('0630ccm1.csv')
ccm_4 = pd.read_csv('0630ccm2.csv')
ccm_data = [ccm_1, ccm_2, ccm_3, ccm_4]

ccmfn_1 = pd.read_csv('0630ccmfn1.csv')
ccmfn_2 = pd.read_csv('0630ccmfn2.csv')
ccmfn_3 = pd.read_csv('0630ccmfn3.csv')
ccmfn_4 = pd.read_csv('0621ccmfn2.csv')
ccmfn_data = [ccmfn_1, ccmfn_2, ccmfn_3, ccmfn_4]

complete_1 = pd.read_csv('0701complete1.csv')
complete_2 = pd.read_csv('0701complete2.csv')
complete_3 = pd.read_csv('0621complete1.csv')
complete_data = [complete_1, complete_2, complete_3]

complfn_1 = pd.read_csv('0706completefn1.csv')
complfn_2 = pd.read_csv('0706completefn2.csv')
complfn_3 = pd.read_csv('0706completefn3.csv')
complfn_4 = pd.read_csv('0706completefn4.csv')
complfn_5 = pd.read_csv('0621completefn1.csv')
complfn_6 = pd.read_csv('0621completefn2.csv')
completefn_data = [complfn_1,complfn_2,complfn_3,complfn_4,complfn_5,complfn_6]

#initial function to filter data 
def clean_data(dataframe, media_type, date, movie_number):
    df = dataframe
    df['Media_type'] = media_type
    df['Unique_id'] = movie_number + media_type + date 
    df = dataframe.drop(columns = ['Convexity_0', 'Number_of_Defects_0',
       'Mean_Defect_Displacement_0', 'Mean_Defect_Area_0',
       'Variance_of_Defect_Area_0', 'Convex_Hull_Center_0',
       'Convex_Hull_Center_1', 'Kurtosis_of_Intensity_0', 'Maximum_intensity_0',
       'Mean_Intensity_0', 'Minimum_intensity_0',
       'Principal_components_of_the_object_0',
       'Principal_components_of_the_object_1',
       'Principal_components_of_the_object_2',
       'Principal_components_of_the_object_3', 'Radii_of_the_object_0',
       'Radii_of_the_object_1', 'Skewness_of_Intensity_0', 'Total_Intensity_0',
       'Variance_of_Intensity_0', 'Average_Branch_Length_0',
       'Number_of_Branches_0', 'Diameter_0', 'Euclidean_Diameter_0',
       'Number_of_Holes_0', 'Center_of_the_Skeleton_0',
       'Center_of_the_Skeleton_1', 'Terminal_1_0', 'Terminal_1_1',
       'Terminal_2_0', 'Terminal_2_1', 'Length_of_the_Skeleton_0',
       'Bounding_Box_Maximum_0', 'Bounding_Box_Maximum_1',
       'Bounding_Box_Minimum_0', 'Bounding_Box_Minimum_1', 'Size_in_pixels_0',
       'Center_of_the_object_0', 'Center_of_the_object_1'], axis = 1)    
    removed_df = df.drop(df[df.mergerLabelId > 0].index)
    removed_df_2 = removed_df.drop(removed_df[removed_df.Object_Area_0 > 4000].index)
    removed_df_3 = removed_df_2.drop(removed_df_2[removed_df_2.Object_Area_0 < 80].index)
    removed_df_4 = removed_df_3.drop(removed_df_3[removed_df_3.trackId == -1].index)
    final_df = removed_df_4
    return final_df

#concat all individual move dfs
def create_master():
    cmaster_df = pd.concat([clean_data(ccm_1, 'CCM', '0701', '1' ),clean_data(ccm_2, 'CCM', '0701', '2'),clean_data(ccm_3, 'CCM', '0630', '1'),
    clean_data(ccm_4, 'CCM', '0630', '2'), clean_data(ccmfn_1, 'CCMFN', '0630', '1'),clean_data(ccmfn_2, 'CCMFN', '0630', '2'),clean_data(ccmfn_3, 'CCMFN', '0630', '3'),
    clean_data(ccmfn_4, 'CCMFN', '0621', '2'),clean_data(complfn_2, 'COMPLFN', '0706', '2'),clean_data(complfn_3, 'COMPLFN', '0706', '3'),
    clean_data(complfn_4, 'COMPLFN', '0706', '4'),clean_data(complfn_5, 'COMPLFN', '0621', '1'),clean_data(complfn_6, 'COMPLFN', '0621', '2'),clean_data(complete_1, 'COMPLETE', '0701', '1'),
    clean_data(complete_3, 'COMPLETE', '0621', '1'), clean_data(complete_2, 'COMPLETE', '0701', '2')])
    return cmaster_df

data = create_master()

In [ ]:



#master function with nested function to calculate distance increments for individual trackids -- while also filtering out trackids that have large distance spikes
def calculateDis1 (U_id):
    condit = (data.Unique_id == U_id) #& (data.trackId == 3)
    run = data[condit]
    dataframe = run.drop(columns = ['labelimageId', 'lineageId','parentTrackId','mergerLabelId','Media_type'])
    dataframe.reset_index(inplace = True)


    def distance(row):
        x1, y1, x2, y2 = row["Object_Center_0"], row["Object_Center_1"], row["X2"], row["Y2"]
        if np.isnan(x2) or np.isnan(y2):
            return 0
        return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    dataframe["X2"] = dataframe.groupby("trackId")["Object_Center_0"].shift(-1)
    dataframe["Y2"] = dataframe.groupby("trackId")["Object_Center_1"].shift(-1)

    dataframe["distance"] = dataframe.apply(distance, axis=1)
    dataframe.groupby("trackId")["distance"].sum()

    m = ~dataframe['distance'].diff().gt(75).groupby(dataframe['trackId']).transform('any')
    filtered_df = dataframe.loc[m, dataframe.columns]

    return filtered_df

#storing output from above function to variables (just for easy scalability if needed)
ccms_1 = calculateDis1('1CCM0701')
ccms_2 = calculateDis1('2CCM0701')
ccms_3 = calculateDis1('2CCM0630')
ccms_4 = calculateDis1('1CCM0630')
ccmfns_1 = calculateDis1('1CCMFN0630')
ccmfns_2 = calculateDis1('2CCMFN0630')
ccmfns_3 = calculateDis1('3CCMFN0630')
ccmfns_4 = calculateDis1('2CCMFN0621')
compls_1 = calculateDis1('1COMPLETE0621')
compls_2 = calculateDis1('2COMPLETE0701')
compls_3 = calculateDis1('1COMPLETE0701')
complfns_1 = calculateDis1('2COMPLFN0706')
complfns_2 = calculateDis1('3COMPLFN0706')
complfns_3 = calculateDis1('2COMPLFN0621')
complfns_4 = calculateDis1('1COMPLFN0621')



#use output of previous function to sum up total distances for each trackid 
def calcfinal(dataframe):
        new_df = dataframe.groupby("trackId")["distance"].sum().reset_index()
        new_df = 
        return new_df

    
ccm_1_dis = calcfinal(ccms_1)
ccm_2_dis = calcfinal(ccms_2)
ccm_3_dis = calcfinal(ccms_3)
ccm_4_dis = calcfinal(ccms_4)
ccmfn_1_dis = calcfinal(ccmfns_1)
ccmfn_2_dis = calcfinal(ccmfns_2)
ccmfn_3_dis = calcfinal(ccmfns_3)
ccmfn_4_dis = calcfinal(ccmfns_4)
compl_1_dis = calcfinal(compls_1)
compl_2_dis = calcfinal(compls_2)
compl_3_dis = calcfinal(compls_3)
complfn_1_dis = calcfinal(complfns_1)
complfn_2_dis = calcfinal(complfns_2)
complfn_3_dis = calcfinal(complfns_3)
complfn_4_dis = calcfinal(complfns_4)

ccm_1_dis[] = calcfinal(ccms_1)
ccm_2_dis[] = calcfinal(ccms_2)
ccm_3_dis[] = calcfinal(ccms_3)
ccm_4_dis[] = calcfinal(ccms_4)
ccmfn_1_dis[] = calcfinal(ccmfns_1)
ccmfn_2_dis[] = calcfinal(ccmfns_2)
ccmfn_3_dis[] = calcfinal(ccmfns_3)
ccmfn_4_dis[] = calcfinal(ccmfns_4)
compl_1_dis[] = calcfinal(compls_1)
compl_2_dis[] = calcfinal(compls_2)
compl_3_dis[] = calcfinal(compls_3)
complfn_1_dis[] = calcfinal(complfns_1)
complfn_2_dis[] = calcfinal(complfns_2)
complfn_3_dis[] = calcfinal(complfns_3)
complfn_4_dis[] = calcfinal(complfns_4)

In [ ]:
#exporting dfs to csv
ccm_1_dis.to_csv('~/Desktop/dfs/ccm_1.csv', index = True)
ccm_2_dis.to_csv('~/Desktop/dfs/ccm_2.csv', index = True)
ccm_3_dis.to_csv('~/Desktop/dfs/ccm_3.csv', index = True)
ccm_4_dis.to_csv('~/Desktop/dfs/ccm_4.csv', index = True)
ccmfn_1_dis.to_csv('~/Desktop/dfs/ccmfn_1.csv', index = True)
ccmfn_2_dis.to_csv('~/Desktop/dfs/ccmfn_2.csv', index = True)
ccmfn_3_dis.to_csv('~/Desktop/dfs/ccm_3.csv', index = True)
ccmfn_4_dis.to_csv('~/Desktop/dfs/ccm_4.csv', index = True)
compl_1_dis.to_csv('~/Desktop/dfs/compl_1.csv', index = True)
compl_2_dis.to_csv('~/Desktop/dfs/compl_2.csv', index = True)
compl_3_dis.to_csv('~/Desktop/dfs/compl_3.csv', index = True)
complfn_1_dis.to_csv('~/Desktop/dfs/complfn_1.csv', index = True)
complfn_2_dis.to_csv('~/Desktop/dfs/complfn_2.csv', index = True)
complfn_3_dis.to_csv('~/Desktop/dfs/complfn_3.csv', index = True)
complfn_4_dis.to_csv('~/Desktop/dfs/complfn_4.csv', index = True)

In [186]:
#function to calculate total frames for each trackid - takes in a movie and spits out list of frames for each trackid
def get_frames(U_id):
    c = data.Unique_id == U_id
    df = data[c]
    dff = calculateDis1(U_id)
    k = sorted(df.trackId.unique())
    mylist = []
    for i, val in enumerate(k):
        condition = dff.trackId == val
        df2 = dff[condition]
        y = len(df2)
        mylist.append(y)
        i+=1
        final_list = [x for x in mylist if not x == 0]
    return final_list

#creating new dataframe column called 'frames' and populating it with the output from get_frames function
ccm_1_dis['frames'] = get_frames('1CCM0701')
ccm_2_dis['frames'] = get_frames('2CCM0701')
ccm_3_dis['frames'] = get_frames('2CCM0630')
ccm_4_dis['frames'] = get_frames('1CCM0630')
ccmfn_1_dis['frames'] = get_frames('1CCMFN0630')
ccmfn_2_dis['frames'] = get_frames('2CCMFN0630')
ccmfn_3_dis['frames'] = get_frames('3CCMFN0630')
ccmfn_4_dis['frames'] = get_frames('2CCMFN0621')
compl_1_dis['frames'] = get_frames('1COMPLETE0621')
compl_2_dis['frames'] = get_frames('2COMPLETE0701')
compl_3_dis['frames'] = get_frames('1COMPLETE0701')
complfn_1_dis['frames'] = get_frames('2COMPLFN0706')
complfn_2_dis['frames'] = get_frames('3COMPLFN0706')
complfn_3_dis['frames'] = get_frames('2COMPLFN0621')
complfn_4_dis['frames'] = get_frames('1COMPLFN0621')

#creating new dataframe column called 'speed' and populating it with the speed in microns per min
ccm_1_dis['speed'] = ((ccm_1_dis.distance / ccm_1_dis.frames) * .65) / 10
ccm_2_dis['speed'] = ((ccm_2_dis.distance / ccm_2_dis.frames) * .65) / 10
ccm_3_dis['speed'] = ((ccm_3_dis.distance / ccm_3_dis.frames) * .65) / 10
ccm_4_dis['speed'] = ((ccm_4_dis.distance / ccm_4_dis.frames) * .65) / 10
ccmfn_1_dis['speed'] = ((ccmfn_1_dis.distance / ccmfn_1_dis.frames) * .65) / 10
ccmfn_2_dis['speed'] = ((ccmfn_2_dis.distance / ccmfn_2_dis.frames) * .65) / 10
ccmfn_3_dis['speed'] = ((ccmfn_3_dis.distance / ccmfn_3_dis.frames) * .65) / 10
ccmfn_4_dis['speed'] = ((ccmfn_4_dis.distance / ccmfn_4_dis.frames) * .65) / 10
compl_1_dis['speed'] = ((compl_1_dis.distance / compl_1_dis.frames) * .65) / 10
compl_2_dis['speed'] = ((compl_2_dis.distance / compl_2_dis.frames) * .65) / 10
compl_3_dis['speed'] = ((compl_3_dis.distance / compl_3_dis.frames) * .65) / 10
complfn_1_dis['speed'] = ((complfn_1_dis.distance / complfn_1_dis.frames) * .65) / 10
complfn_2_dis['speed'] = ((complfn_2_dis.distance / complfn_2_dis.frames) * .65) / 10
complfn_3_dis['speed'] = ((complfn_3_dis.distance / complfn_3_dis.frames) * .65) / 10
complfn_4_dis['speed'] = ((complfn_4_dis.distance / complfn_4_dis.frames) * .65) / 10

#creating new dataframe column called 'media type' and populating it with the coordinated media type 
ccm_1_dis['Media_type'] = 'CCM-1'
ccm_2_dis['Media_type'] = 'CCM-1'
ccm_3_dis['Media_type'] = 'CCM-1'
ccm_4_dis['Media_type'] = 'CCM-1'
ccmfn_1_dis['Media_type'] = 'CCM-1 + FN'
ccmfn_2_dis['Media_type'] = 'CCM-1 + FN'
ccmfn_3_dis['Media_type'] = 'CCM-1 + FN'
ccmfn_4_dis['Media_type'] = 'CCM-1 + FN'
compl_1_dis['Media_type'] = 'COMPLETE'
compl_2_dis['Media_type'] = 'COMPLETE'
compl_3_dis['Media_type'] = 'COMPLETE'
complfn_1_dis['Media_type'] = 'COMPLETE + FN'
complfn_2_dis['Media_type'] = 'COMPLETE + FN'
complfn_3_dis['Media_type'] = 'COMPLETE + FN'
complfn_4_dis['Media_type'] = 'COMPLETE + FN'

#creating new dataframe column called 'movie' and populating it with the coordinates U-id
ccm_1_dis['Movie'] = '1CCM0701'
ccm_2_dis['Movie'] = '2CCM0701'
ccm_3_dis['Movie'] = '2CCM0630'
ccm_4_dis['Movie'] = '1CCM0630'
ccmfn_1_dis['Movie'] = '1CCMFN0630'
ccmfn_2_dis['Movie'] = '2CCMFN0630'
ccmfn_3_dis['Movie'] = '3CCMFN0630'
ccmfn_4_dis['Movie'] = '2CCMFN0621'
compl_1_dis['Movie'] = '1COMPLETE0621'
compl_2_dis['Movie'] = '2COMPLETE0701'
compl_3_dis['Movie'] = '1COMPLETE0701'
complfn_1_dis['Movie'] = '2COMPLFN0706'
complfn_2_dis['Movie'] = '3COMPLFN0706'
complfn_3_dis['Movie'] = '2COMPLFN0621'
complfn_4_dis['Movie'] = '1COMPLFN0621'

#concat all new dfs 
migration_mas = pd.concat([ccm_1_dis, ccm_2_dis,ccm_3_dis,ccm_4_dis, ccmfn_1_dis, ccmfn_2_dis, ccmfn_3_dis, ccmfn_4_dis, 
                          compl_1_dis, compl_2_dis, compl_3_dis, complfn_1_dis, complfn_2_dis, complfn_3_dis, complfn_4_dis])


migration_mas.to_csv('~/Desktop/REU/migration_mas.csv', index = True)

In [ ]:
def calc_speed(sample):
    distance = sample.distance
    frames = sample.frames
    speed = ((distance/frames)*.650) / 10
    return speed

In [249]:
def get_point_a(data, u_id):
    condition = data.Unique_id == u_id
    dataframe = data[condition]
    k = sorted(data.trackId.unique())
    mylist = []
    for i, val in enumerate(k):
        condi = data.trackId == val
        df2 = data[condi]
        y = df2.Object_Center_0.iloc[0] 
        mylist.append(y)
    return mylist
get_point_a(ccms_1,'1CCM0701')
    
def get_point_b(data, u_id):
    condition = data.Unique_id == u_id
    dataframe = data[condition]
    k = sorted(data.trackId.unique())
    mylist = []
    for i, val in enumerate(k):
        condi = data.trackId == val
        df2 = data[condi]
        y = df2.Object_Center_0.iloc[-1] 
        mylist.append(y)
    return mylist
get_point_b(ccms_1,'1CCM0701')

def get_point_b_y(data, u_id):
    condition = data.Unique_id == u_id
    dataframe = data[condition]
    k = sorted(data.trackId.unique())
    mylist = []
    for i, val in enumerate(k):
        condi = data.trackId == val
        df2 = data[condi]
        y = df2.Object_Center_1.iloc[-1] 
        mylist.append(y)
    return mylist
get_point_b(ccms_1,'1CCM0701')

def get_point_a_y(data, u_id):
    condition = data.Unique_id == u_id
    dataframe = data[condition]
    k = sorted(data.trackId.unique())
    mylist = []
    for i, val in enumerate(k):
        condi = data.trackId == val
        df2 = data[condi]
        y = df2.Object_Center_1.iloc[-1] 
        mylist.append(y)
    return mylist
get_point_b(ccms_1,'1CCM0701')





def calc_persistence(datafram):
    point_2 = datafram.point_b
    point_1 = datafram.point_a
    persistence = (point_b - point_a)
    return persistence

ccm_1_dis['point_a'] = get_point_a(ccms_1,'1CCM0701')
ccm_2_dis['point_a'] = get_point_a(ccms_2,'2CCM0701')
ccm_3_dis['point_a'] = get_point_a(ccms_3,'2CCM0630')
ccm_4_dis['point_a'] = get_point_a(ccms_4,'1CCM0630')
ccmfn_1_dis['point_a'] = get_point_a(ccmfns_1,'1CCMFN0630')
ccmfn_2_dis['point_a'] = get_point_a(ccmfns_2,'2CCMFN0630')
ccmfn_3_dis['point_a'] = get_point_a(ccmfns_3,'3CCMFN0630')
ccmfn_4_dis['point_a'] = get_point_a(ccmfns_4,'2CCMFN0621')
compl_1_dis['point_a'] = get_point_a(compls_1,'1COMPLETE0621')
compl_2_dis['point_a'] = get_point_a(compls_2,'2COMPLETE0701')
compl_3_dis['point_a'] = get_point_a(compls_3,'1COMPLETE0701')
complfn_1_dis['point_a'] = get_point_a(complfns_1,'2COMPLFN0706')
complfn_2_dis['point_a'] = get_point_a(complfns_2,'3COMPLFN0706')
complfn_3_dis['point_a'] = get_point_a(complfns_3,'2COMPLFN0621')
complfn_4_dis['point_a'] = get_point_a(complfns_4,'1COMPLFN0621')

ccm_1_dis['point_b'] = get_point_b(ccms_1,'1CCM0701')
ccm_2_dis['point_b'] = get_point_b(ccms_2,'2CCM0701')
ccm_3_dis['point_b'] = get_point_b(ccms_3,'2CCM0630')
ccm_4_dis['point_b'] = get_point_b(ccms_4,'1CCM0630')
ccmfn_1_dis['point_b'] = get_point_b(ccmfns_1,'1CCMFN0630')
ccmfn_2_dis['point_b'] = get_point_b(ccmfns_2,'2CCMFN0630')
ccmfn_3_dis['point_b'] = get_point_b(ccmfns_3,'3CCMFN0630')
ccmfn_4_dis['point_b'] = get_point_b(ccmfns_4,'2CCMFN0621')
compl_1_dis['point_b'] = get_point_b(compls_1,'1COMPLETE0621')
compl_2_dis['point_b'] = get_point_b(compls_2,'2COMPLETE0701')
compl_3_dis['point_b'] = get_point_b(compls_3,'1COMPLETE0701')
complfn_1_dis['point_b'] = get_point_b(complfns_1,'2COMPLFN0706')
complfn_2_dis['point_b'] = get_point_b(complfns_2,'3COMPLFN0706')
complfn_3_dis['point_b'] = get_point_b(complfns_3,'2COMPLFN0621')
complfn_4_dis['point_b'] = get_point_b(complfns_4,'1COMPLFN0621')

ccm_1_dis['point_a_y'] = get_point_a_y(ccms_1,'1CCM0701')
ccm_2_dis['point_a_y'] = get_point_a_y(ccms_2,'2CCM0701')
ccm_3_dis['point_a_y'] = get_point_a_y(ccms_3,'2CCM0630')
ccm_4_dis['point_a_y'] = get_point_a_y(ccms_4,'1CCM0630')
ccmfn_1_dis['point_a_y'] = get_point_a_y(ccmfns_1,'1CCMFN0630')
ccmfn_2_dis['point_a_y'] = get_point_a_y(ccmfns_2,'2CCMFN0630')
ccmfn_3_dis['point_a_y'] = get_point_a_y(ccmfns_3,'3CCMFN0630')
ccmfn_4_dis['point_a_y'] = get_point_a_y(ccmfns_4,'2CCMFN0621')
compl_1_dis['point_a_y'] = get_point_a_y(compls_1,'1COMPLETE0621')
compl_2_dis['point_a_y'] = get_point_a_y(compls_2,'2COMPLETE0701')
compl_3_dis['point_a_y'] = get_point_a_y(compls_3,'1COMPLETE0701')
complfn_1_dis['point_a_y'] = get_point_a_y(complfns_1,'2COMPLFN0706')
complfn_2_dis['point_a_y'] = get_point_a_y(complfns_2,'3COMPLFN0706')
complfn_3_dis['point_a_y'] = get_point_a_y(complfns_3,'2COMPLFN0621')
complfn_4_dis['point_a_y'] = get_point_a_y(complfns_4,'1COMPLFN0621')

ccm_1_dis['point_b_y'] = get_point_b_y(ccms_1,'1CCM0701')
ccm_2_dis['point_b_y'] = get_point_b_y(ccms_2,'2CCM0701')
ccm_3_dis['point_b_y'] = get_point_b_y(ccms_3,'2CCM0630')
ccm_4_dis['point_b_y'] = get_point_b_y(ccms_4,'1CCM0630')
ccmfn_1_dis['point_b_y'] = get_point_b_y(ccmfns_1,'1CCMFN0630')
ccmfn_2_dis['point_b_y'] = get_point_b_y(ccmfns_2,'2CCMFN0630')
ccmfn_3_dis['point_b_y'] = get_point_b_y(ccmfns_3,'3CCMFN0630')
ccmfn_4_dis['point_b_y'] = get_point_b_y(ccmfns_4,'2CCMFN0621')
compl_1_dis['point_b_y'] = get_point_b_y(compls_1,'1COMPLETE0621')
compl_2_dis['point_b_y'] = get_point_b_y(compls_2,'2COMPLETE0701')
compl_3_dis['point_b_y'] = get_point_b_y(compls_3,'1COMPLETE0701')
complfn_1_dis['point_b_y'] = get_point_b_y(complfns_1,'2COMPLFN0706')
complfn_2_dis['point_b_y'] = get_point_b_y(complfns_2,'3COMPLFN0706')
complfn_3_dis['point_b_y'] = get_point_b_y(complfns_3,'2COMPLFN0621')
complfn_4_dis['point_b_y'] = get_point_b_y(complfns_4,'1COMPLFN0621')







ccm1pa = ccm_1_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
ccm2pa = ccm_2_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
ccm3pa = ccm_3_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
ccm4pa = ccm_4_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
ccmfn1pa = ccmfn_1_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
ccmfn2pa = ccmfn_2_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
ccmfn3pa = ccmfn_3_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
ccmfn4pa = ccmfn_4_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
compl1pa = compl_1_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
compl2pa = compl_2_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
compl3pa = compl_3_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
complfn1pa = complfn_1_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
complfn2pa = complfn_2_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
complfn3pa = complfn_3_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])
complfn4pa = complfn_4_dis.drop(columns = ['trackId','frames','speed','Media_type','Movie', 'distance'])

In [ ]:
actual_master = pd.concat([loop(ccm_1_dis), loop(ccm_2_dis), loop(ccm_3_dis), loop(ccm_4_dis), loop(ccmfn_1_dis), loop(ccmfn_2_dis), loop(ccmfn_3_dis), loop(ccmfn_4_dis),
                               loop(compl_1_dis), loop(compl_2_dis), loop(compl_3_dis), loop(complfn_1_dis), loop(complfn_2_dis), loop(complfn_3_dis), loop(complfn_4_dis)])
                           

In [316]:
completed_master = migration_mas.join(persis_mas)

actual_master.to_csv('~/Desktop/REU/actual_mas.csv', index = True)

In [237]:
persis_mas = pd.concat([ccm1pa, ccm2pa, ccm3pa, ccm4pa, ccmfn1pa, ccmfn2pa, ccmfn3pa, ccmfn4pa, compl1pa, compl2pa, compl3pa, complfn1pa, complfn2pa, complfn3pa, complfn4pa ])

In [312]:
#function with nested loop to calc persistence 
def loop(dataset):
    i = 0 
    length = len(dataset)
    mylist = []
    while i < length: 
        point_a = dataset.point_a[i]
        point_b = dataset.point_b[i]
        point_b_y = dataset.point_b_y[i]
        point_a_y = dataset.point_a_y[i]
        distance = dataset.distance[i]
        y = math.sqrt((dataset.point_b[i] - dataset.point_a[i])**2 + (dataset.point_b_y[i] - dataset.point_a_y[i])**2) / dataset.distance[i]
        mylist.append(y)
        i+= 1
    dataset['PERS'] = mylist
    return dataset